In [1]:
import pandas as pd
import numpy as np
import string

# Read the content of the file using ISO-8859-1 encoding
with open('./old_par.txt', 'r', encoding='ISO-8859-1') as f:
    content = f.read()


# Write the content back to the file using UTF-8 encoding
with open('./old_par.txt', 'w', encoding='utf-8') as f:
    f.write(content)

In [2]:
# Read the file into a DataFrame, ignoring lines starting with '#'
old_par= pd.read_csv('./data/par_text.txt', comment='!', delimiter='\t', header= None, index_col=0)

In [3]:
# Read the file into a DataFrame, ignoring lines starting with '#'
par = pd.read_csv('./data/par_text.txt', comment='!', delimiter='\t', header= None, index_col=0)

In [4]:
# read cal params
cal_params= pd.read_csv('./cal_param.txt', delimiter='\t',header= None,index_col=0)

In [5]:
cal_params

,1,2,3,4,5,6
0,,,,,,
ttmp,2.909494,-2.670,3.00,none,none,none
cmlt,4.220418,1.000,10.00,none,none,none
cmrad,5.000000,1.000,10.00,none,none,none
kc,1.276205,0.600,1.30,none,none,none
alb,0.200000,0.100,0.50,none,none,none
srrcs,0.175497,0.010,0.20,none,none,none
rrcs1,0.073282,0.050,0.60,none,none,none
rrcs2,0.002159,0.001,0.45,none,none,none
bcosby,13.122820,1.000,20.00,none,none,none


In [6]:
# Convert index of cal_params to string
cal_params.index = cal_params.index.astype(str)

# Convert index of par to string
par.index = par.index.astype(str)

In [7]:
# Create an empty list to record strings being written
strings_written = []

In [8]:
# Define a mapping of column names to letters
column_letters = {col: letter for col, letter in zip(par.columns, string.ascii_uppercase)}

# Iterate through the index of the par DataFrame
for par_index in par.index:
    # Check if the index exists in cal_params
    if par_index in cal_params.index:
        # Get the row index of the value in cal_params DataFrame
        cal_params_row_index = cal_params.index.get_loc(par_index)
        # Iterate through the columns of par DataFrame
        for col in par.columns:
            # Update the value in the par DataFrame
            par_value = par.at[par_index, col]
            # Stop writing if a value of nan is found
            if pd.isna(par_value):
                break
            else:
                # Get the corresponding letter for the column name
                letter = column_letters[col]
                new_string = f'___{par_index}{letter}'  # Construct the new string with letter instead of {col}
                strings_written.append(new_string)  # Append the new string to the list
                par.at[par_index, col] = new_string  # Update the value in the par DataFrame

In [9]:
ostin = pd.DataFrame(index=strings_written, columns=cal_params.columns)

In [10]:
ostin

,1,2,3,4,5,6
___ttmpA,NaN,NaN,NaN,NaN,NaN,NaN
___ttmpB,NaN,NaN,NaN,NaN,NaN,NaN
___ttmpC,NaN,NaN,NaN,NaN,NaN,NaN
___ttmpD,NaN,NaN,NaN,NaN,NaN,NaN
___ttmpE,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...
___mperc2P,NaN,NaN,NaN,NaN,NaN,NaN
___mperc2Q,NaN,NaN,NaN,NaN,NaN,NaN
___rrcs3A,NaN,NaN,NaN,NaN,NaN,NaN
___dampA,NaN,NaN,NaN,NaN,NaN,NaN


In [11]:
# Iterate through the rows of the cal_params DataFrame
for cal_params_index, cal_params_row in cal_params.iterrows():
    # Iterate through the index of ostin
    for ostin_index in ostin.index:
        # Check if the string in the index of ostin contains the string in the index of cal_params
        if cal_params_index in ostin_index:
            # Append the row from cal_params to the corresponding index in ostin
            ostin.loc[ostin_index] = cal_params_row.values

In [12]:
ostin

,1,2,3,4,5,6
___ttmpA,2.909494,-2.67,3.0,none,none,none
___ttmpB,2.909494,-2.67,3.0,none,none,none
___ttmpC,2.909494,-2.67,3.0,none,none,none
___ttmpD,2.909494,-2.67,3.0,none,none,none
___ttmpE,2.909494,-2.67,3.0,none,none,none
...,...,...,...,...,...,...
___mperc2P,60.0,5.0,120.0,none,none,none
___mperc2Q,60.0,5.0,120.0,none,none,none
___rrcs3A,0.080346,0.0,0.1,none,none,none
___dampA,0.900375,0.1,0.95,none,none,none


In [13]:
# Custom header and footer strings
top_half = """#Basic Configuration for Ostrich Program

#Essential variables
ProgramType ParallelDDS
ModelExecutable ./kge_calibrate.py
ModelSubdir mod_
ObjectiveFunction GCOP
OstrichWarmStart no

#Template File Configuration
BeginFilePairs
data/par_paired.txt ; data/par.txt
EndFilePairs

BeginExtraFiles
EndExtraFiles

BeginExtraDirs
data
EndExtraDirs


BeginParams"""

In [14]:
bottom_half= """EndParams

#Output variables to analyze
BeginResponseVars
#name   filename            keyword         line    col     token
KGE ./average_kge.txt ; OST_NULL 0 1 ' '
EndResponseVars

BeginGCOP
CostFunction KGE
PenaltyFunction APM
EndGCOP

#DDS algorithm setup
BeginParallelDDSAlg
PerturbationValue 0.2
MaxIterations 50000
UseInitialParamValues
EndParallelDDSAlg
"""


In [15]:
# Write the DataFrame to a text file with header and footer strings
with open('./ostIn.txt', 'w') as file:
    file.write(top_half + '\n')
    ostin.to_csv(file, header=False, sep=' ')  # You can change the separator if needed
    file.write(bottom_half)

In [16]:
# Save par DataFrame as a tab-separated text file
par.to_csv('./data/par_paired.txt', sep='\t',header=False)

In [17]:
# Save par DataFrame as a tab-separated text file
old_par.to_csv('./data/par.txt', sep='\t',header=False)